In [ ]:
def printSelect(df: DataFrame, dfName: str="", dataType: str="", alias: str="", dfSuffix: str="", updateName: str="", sort: str="") -> str:
    """
    Function:

        Print a block of code in the form of a select()-statement that contains:
            - the dataframe's original and new name (as defined by the new suffix)
            - the dataframe's original and new column names, alphabetically ordered or not

    Parameters:

        - df: the dataframe from which you want to change its name and column names
        - df_name: name of the dataframe you want to print
        - dataType:
            - "withDataType": adds the datatype in a commented section after the string (e.g. # string)
            - "withoutDataType": without the datatype in a commented section after the string
        - alias:
            - "withAlias": with alias() function
            - "withoutAlias": without alias() function
        - dfSuffix: defines the suffix of the new dataframe name (e.g., "dataframeT2" if the original dataframe's name was "dataframeT1")
        - updateName: defines the new column names and has three values ("withoutPrefix", "keepPrefix", and "updatePrefix") 
            - "withoutPrefix" removes the previous dataframe-name from the column names if it was present
            - "keepPrefix" keeps the previous dataframe-name as a prefix in the new column names
            - "updatePrefix" adds the new dataframe-name as a prefix to the column names
        - sort: sorts the columns alphabetically or not
            - "sorted": sorts columns alphabetically
            - "notsorted": leaves columns ordered as is
    
    Output:

        Return: block of code containing the dataframe's original and new column names in the form of a select()-statement.

    """
    
    # Import
    import os
    import re
    
    # Get name of the table
    OldTableName = dfName

    # Set the new table name
    NewTableName = OldTableName

    # Remove "Parquet" or "Delta" suffix from the old table name
    if OldTableName.endswith("Parquet"):
        NewTableName = OldTableName[:-len("Parquet")]
    elif OldTableName.endswith("Delta"):
        NewTableName = OldTableName[:-len("Delta")]


    # String consisting of whitespace
    tabSpace = "    "

    # Create first line of the select-statement
    ls = os.linesep
    sCols = f"{NewTableName} = {dfName} \{ls}{tabSpace}.select({ls}"

    # Get columnnames and datatypes
    dt = df.dtypes

    # Sort columns alphabetically or not
    if sort == "sorted":
        dt = sorted(dt)
    elif sort == "notsorted" or sort == "":
        dt = dt

    # Loop through every item in dt (contains the name and datatype of the column)
    for i, (cName, dType) in enumerate(dt):

        # Get tablename and columnname + create first part of string
        TableName, ColumnName = cName.split('_', 1) if '_' in cName else ("", cName)
        OldColumnName = f"{TableName}_{ColumnName}" if TableName else f"{ColumnName}"

        # Create string with the old columnname and the new columnname
        if alias == "withAlias":

            # Set the new columnname (may include the old or new tablename)
            if updateName == "updatePrefix":
                NewColumnName = f"{NewTableName}.{ColumnName}"
            elif updateName == "keepPrefix" and TableName != "":
                NewColumnName = f"{TableName}.{ColumnName}" if TableName else f"{ColumnName}"
            elif updateName == "keepPrefix" and TableName == "":
                NewColumnName = ColumnName
            elif updateName == "withoutPrefix":
                NewColumnName = ColumnName
            elif updateName == "":
                NewColumnName = ColumnName

            # Update the columnnames
            if dataType == "withoutDataType" or dataType == "":
                sCols += f"{tabSpace}{tabSpace}col(\"{OldColumnName}\").alias(\"{NewColumnName}\"),{ls}"
            elif dataType == "withDataType":
                sCols += f"{tabSpace}{tabSpace}col(\"{OldColumnName}\").alias(\"{NewColumnName}\"), # {dType}{ls}"

        # Create string with the old columnname
        elif alias == "withoutAlias" or alias == "":
            if dataType == "withoutDataType" or dataType == "":
                sCols += f"{tabSpace}{tabSpace}col(\"{OldColumnName}\"),{ls}"
            elif dataType == "withDataType":
                sCols += f"{tabSpace}{tabSpace}col(\"{OldColumnName}\"), # {dType}{ls}"

    # Add bracket to the final line
    sCols = sCols + f"{tabSpace})"
    
    # Return the columnnames in the form of a select()-statement
    return print(sCols)

# Example usage
# printSelect(df_BC_CustomersParquet, "df_BC_CustomersParquet", "withDataType", "withoutAlias", "", "keepPrefix", "notsorted")